In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

import os
import re
import pandas as pd
import numpy as np
from typing import List, Tuple, Dict, Union
from typing import Iterable as Iter
from typing import Callable
from sklearn.base import BaseEstimator, TransformerMixin

In [32]:
acts = 'parquet_labelbox/labelbox_atos_aviso_licitacao.parquet'
entities = 'parquet_labelbox/labelbox_entidades_aviso_licitacao.parquet'
texts = 'parquet_labelbox/tokenized_texts_aviso_licitacao.parquet'

df1 = pd.read_parquet(acts)
df2 = pd.read_parquet(entities)
df3 = pd.read_parquet(texts)

In [33]:
df2['ner_token'].unique()

array(['ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'PROCESSO', 'DATA_ABERTURA', 'TIPO_OBJ',
       'VALOR_ESTIMADO', 'CODIGO_SISTEMA_COMPRAS', 'SISTEMA_COMPRAS'],
      dtype=object)

In [34]:
tokens = df2['ner_token'].unique()

for token in tokens:
    df1[token] = np.nan

In [35]:
def clean_text_by_word(text):
    a = "\n".join([l for l in text.split("\n") if l != ""])
    words = a.replace("\n", " ").split(" ")
    words = [w for w in words if w != ""]
    
    m_words = []
    dash_cut = False

    for i in range(len(words)):
        word = words[i]

        if (word[-1] == "-") and (i+1)<len(words):
            word = word[:-1] + words[i+1]
            i += 1

        m_words.append(word)
        
    return " ".join(m_words)

In [36]:
for i in range(len(df2)):
    for j in range(len(df1)):
        if df2['arquivo_rast'][i] == df1['arquivo_rast'][j]:
            entitie = re.sub('xxbcet ?|xxbcet ?|xxeob ?|xxbob ?|xxecet ?', '', clean_text_by_word(df2['entidade'][i]).replace("\r", ""))
            df1.loc[j, df2['ner_token'][i]] = entitie


### IOB

In [5]:
DEFAULT_TOKENIZER = nltk.RegexpTokenizer(r"\w+").tokenize
class Tokenizer(TransformerMixin, BaseEstimator):
    """ Class to apply tokenizer to pandas DataFrame.
    """
    def __init__(self, tokenizer=DEFAULT_TOKENIZER):
        self.tokenizer = tokenizer


    def __call__(self, X, **kw_params):
        return self.tokenizer(X, **kw_params)


    def fit(self, X, y=None, **fit_params):
        return self


    def transform(self, X, **kw_params):
        if not isinstance(X, pd.Series):
            print("[preprocess.Tokenizer.transform] TYPE:", type(X))
            print('X:::: ', X)
            X = pd.Series(X)
        return X.map(self)


from sklearn.base import BaseEstimator, TransformerMixin

class IOBifyer(TransformerMixin, BaseEstimator):


    @staticmethod
    def find_entity(row, token, ignore_idx=0,
        tokenizer=DEFAULT_TOKENIZER):
        # TODO: aceitar opção de offset, para não ter tennhum tipo de problema
        """Searches for named entities on columns, except by ignore_idx-columns.

        ignore_idx: int indicating which column has
                    the TEXT where the named entity were extracted from
        """
        for idx, column in enumerate(row.keys()):
            if idx == ignore_idx:
                continue
            if isinstance(row[column], str) and \
                token == tokenizer(row[column])[0]:
                return column

        return None


    @staticmethod
    def generate_IOB_labels(row, idx, tokenizer, dbg={}):
        """[summary]

        Args:
            row ([pd.Series]): [pandas series having act text and entities text]
            idx ([int]): [index such that `row[idx]` has the whole act]
            tokenizer ([Callable]): [function to use to tokenize `row[idx]`]
            dbg (dict, optional): [dictionay for debug purposes]. Defaults to {}.

        Returns:
            [Iter[Iter[str]]]: [matrix of IOB labels]
        """
        labels = []
        entity_started = False
        text = row.iloc[idx]
        for token in tokenizer(text):                         # Itera sobre cada token da anotação do ato.
            if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
                entity = IOBifyer.find_entity(row, token, idx)                 # Busca o token atual no primeiro token de todos os campos do df.
                if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                    entity_started = True
                    token_index = 1
                    labels.append('B-' + entity)
                else:
                    labels.append('O')
            else:     # Caso uma entidade já tenha sido identificada
                if token_index < len(tokenizer(row[entity])) and \
                    token == tokenizer(row[entity])[token_index]:
                    # Checa se o próximo token pertence à entidade
                    # e se o tamanho da entidade chegou ao fim.
                    labels.append('I-' + entity)
                    # Se a entidade ainda possui tokens e a comparação foi bem
                    # sucedida adicione o label I.
                    token_index += 1
                    if token_index >= len(tokenizer(row[entity])):
                        entity_started = False
                else:
                    # Se o token n for igual ou a entidade chegou ao fim.
                    entity_started = False
                    labels.append('O')
        if labels[0] != 'O':
            dbg['l'] = dbg.get('l', []) + [(row, idx)]

        return labels


    @staticmethod
    def dump_iob(tokens_mat, labels_mat, path='dump.txt',
                            sep=' X X ', sent_sep='\n',):
        """This method dumps the token matrix according its IOB labels.

        For debug purposes, a list of list of pairs (token, label) is returned.
        Args:
            tokens_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to tokens]
            labels_mat ([Iter[Iter[str]]]): [matrix of strings corresponding to IOB labels]
            path (str, optional): [Path to dump text file]. Defaults to 'dump.txt'.
            sep (str, optional): [description]. Defaults to ' X X '.
            sent_sep (str, optional): [description]. Defaults to '\n'.

        Returns:
            [List[LIst[Tuple(str, str)]]]: [list of list of pairs (token, label), as dumped. For debug purposes.]
        """
        dbg_mat = []
        if isinstance(path, Path):
            path = path.as_posix()
        if '/' in path:
            os.makedirs('/'.join(path.split('/')[:-1]), exist_ok=True)

        with open(path, 'w') as fp:
            for tokens_lis, labels_lis in zip(tokens_mat, labels_mat):
                dbg_mat.append([])
                for token, label in zip(tokens_lis, labels_lis):
                    dbg_mat[-1].append((token, label))
                    fp.write(f"{token}{sep}{label}\n")
                fp.write(sent_sep)
        return dbg_mat


    def __init__(self, column='act_column',
        tokenizer=DEFAULT_TOKENIZER):
        self.column = column
        self.tokenizer = tokenizer
        self.dbg = {}


    def fit(self, X=None, y=None, **fit_params):
        return self


    def transform(self, df):
        if not isinstance(df, pd.DataFrame):
            raise TypeError(f"`df` expected to be a pd.DataFrame. Got {type(df)}")
        if df.empty:
            print("[core.preprocess]Warning: empty DataFrame. There won't be ioblabels.")
            return pd.Series()

        idx = self.column if isinstance(self.column, int) else  \
                df.columns.get_loc(self.column)
        labels_row = []
        for index, row in df.iterrows():
            try:
                labels_row.append(
                    IOBifyer.generate_IOB_labels(
                        row, idx, self.tokenizer, self.dbg
                    )
                )
            except Exception as e:
                print("problem iobifyin row:", row)
                raise e
        return pd.Series(labels_row)


In [40]:
df1.columns

Index(['arquivo_rast', 'text', 'ato', 'dodf', 'treated_text',
       'ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'PROCESSO', 'DATA_ABERTURA', 'TIPO_OBJ',
       'VALOR_ESTIMADO', 'CODIGO_SISTEMA_COMPRAS', 'SISTEMA_COMPRAS'],
      dtype='object')

In [41]:
data = df1[['ORGAO_LICITANTE', 'NUM_LICITACAO', 'OBJ_LICITACAO',
       'MODALIDADE_LICITACAO', 'PROCESSO', 'DATA_ABERTURA', 'TIPO_OBJ',
       'VALOR_ESTIMADO', 'CODIGO_SISTEMA_COMPRAS', 'SISTEMA_COMPRAS', 'treated_text',]]

In [42]:
data.drop(166, inplace=True)
data = data.reset_index(drop=True)

C:\Users\ianfi\AppData\Local\Temp\ipykernel_23900\1869218167.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(166, inplace=True)


In [43]:
iob = IOBifyer(column='treated_text')
r = iob.transform(data)

In [44]:
r

0      [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
1      [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
2      [B-MODALIDADE_LICITACAO, I-MODALIDADE_LICITACA...
3      [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
4      [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
                             ...                        
229    [B-MODALIDADE_LICITACAO, O, B-NUM_LICITACAO, I...
230    [B-MODALIDADE_LICITACAO, O, B-NUM_LICITACAO, I...
231    [O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...
232    [O, O, O, O, O, O, B-PROCESSO, I-PROCESSO, I-P...
233    [O, O, O, B-MODALIDADE_LICITACAO, I-MODALIDADE...
Length: 234, dtype: object

In [45]:
data.shape, r.shape

((234, 11), (234,))

In [46]:
data["IOB"] = np.nan

In [47]:
for i in range(len(data)):
    data.loc[i, "IOB"] = ' '.join(r[i])

In [48]:
data['treated_text'][0]

'AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/2021 Processo: 00052-00021170/2020-47. OBJETO: Contratação de Concessionária Autorizada pela Montadora RENA ULT, localizada no Distrito Federal, para prestação de serviços de forma contínua durante o período de garantia, para as manutenções preventivas e corretivas, com fomecimento e aplicação de peças e acessórios genuínos, lubrificantes, óleos, aditivos, higienizadores, alinhamento e balanceamento para 40 (quarenta) viaturas marca RENAULT, modelo Importado Logan ZEN 16MT, anos/modelos 2020/2021, 1.6 flex (gasolina/álcool), pertencentes à frota da Polícia Civil do Distrito Federal., conforme especificações e condições estabelecidas no termo de referência constante do Anexo I do Edital. TIPO: Menor Preço. Valor estimado da licitação: R$ 826.897,20 (oitocentos e vinte e seis mil, oitocentos e noventa e sete reais e vinte centavos). Natureza de Despesa: 33.90.30 e 33.90.39, Fonte 100. Programa de Trabalho 28.845.0903.00NR.0053 Manutenção da Pol

In [49]:
len(DEFAULT_TOKENIZER(data['treated_text'][0]))

273

In [50]:
data['IOB'][0]

'O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM_LICITACAO O B-PROCESSO I-PROCESSO I-PROCESSO I-PROCESSO O B-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I

In [51]:
len(data['IOB'][0].split())

273

In [52]:
df_iob = data[['treated_text', 'IOB']]

In [53]:
df_iob.to_parquet('iob_aviso_licitacao.parquet')

In [2]:
df = pd.read_parquet('iob_aviso_licitacao.parquet')
df

,treated_text,IOB
0,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 01/202...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
1,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO Nº 50/201...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
2,PREGÃO ELETRÔNICO Nº 14/2021 Processo: 092.041...,B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO ...
3,AVISO DE LICITAÇÃO PREGÃO ELETRÔNICO (SRP) Nº ...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
4,AVISO DE LICITAÇÃO TOMADA DE PREÇOS Nº 144/200...,O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICI...
...,...,...
229,PREGÃO Nº 442/2004 SUCOM/SEF/DF Objeto: Aquisi...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
230,PREGÃO Nº 451/2004 SUCOM/SEF/DF Objeto: Aquisi...,B-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM...
231,AVISO DE LICITAÇÃO A COMPANHIA ENERGÉTICA DE B...,O O O O O O O O O O O O O O O O O O O O O O O ...
232,AVISO DE LICITAÇÃO PROCESSO N.º 053.000.918/20...,O O O O O O B-PROCESSO I-PROCESSO I-PROCESSO I...


In [3]:
df['IOB'][0]

'O O O B-MODALIDADE_LICITACAO I-MODALIDADE_LICITACAO O B-NUM_LICITACAO I-NUM_LICITACAO O B-PROCESSO I-PROCESSO I-PROCESSO I-PROCESSO O B-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I-OBJ_LICITACAO I